In [1]:
import sqlite3
db = sqlite3.connect('CL.db')
cur = db.cursor()

In [2]:
import os, re
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm_notebook as tqdm
import random
from lxml.html import fromstring
import time
import pandas as pd

ua = UserAgent(verify_ssl=False)

In [3]:
from IPython.display import clear_output

# Links

In [ ]:
<p itemprop="description">Рассмотрен ассортимент комбинированной группы <span class="hl to-search">корсетных изделий</span>. Для изготовления грации предложены различные современные материалы и нити для их изготовления, представлена их характеристика. Подробно рассмотрено применение подкладочных и клеевых прокладочных, скрепляющих материалов, фурнитуры, видов отделок для изготовления женской грации.</p>

In [9]:
session = requests.session()
headers = {'User-Agent': ua.random}
r = session.get('https://cyberleninka.ru', headers=headers)

html = r.text
soup = BeautifulSoup(html, 'lxml')

fields = {}

for i in soup.find_all('ul', attrs={'class':'grnti'}):
    for j in i.find_all('a'):
        fields[j.text] = j['href']

fields

{'Автоматика': '/article/c/avtomatika-vychislitelnaya-tehnika',
 'Аквакультура': '/article/c/rybnoe-hozyaystvo-akvakultura',
 'Архитектура': '/article/c/stroitelstvo-arhitektura',
 'Астрономия': '/article/c/astronomiya',
 'Атеизм': '/article/c/religiya-ateizm',
 'Биология': '/article/c/biologiya',
 'Биотехнология': '/article/c/biotehnologiya',
 'Бытовое обслуживание': '/article/c/zhilischno-kommunalnoe-hozyaystvo-domovodstvo-bytovoe-obsluzhivanie',
 'Внешняя торговля': '/article/c/vneshnyaya-torgovlya',
 'Внутренняя торговля': '/article/c/vnutrennyaya-torgovlya-turistsko-ekskursionnoe-obsluzhivanie',
 'Водное хозяйство': '/article/c/vodnoe-hozyaystvo',
 'Военное дело': '/article/c/voennoe-delo',
 'Вычислительная техника': '/article/c/avtomatika-vychislitelnaya-tehnika',
 'География': '/article/c/geografiya',
 'Геодезия': '/article/c/geodeziya-kartografiya',
 'Геология': '/article/c/geologiya',
 'Геофизика': '/article/c/geofizika',
 'Горное дело': '/article/c/gornoe-delo',
 'Государство

In [4]:
import random

In [4]:
from pymystem3 import Mystem
m = Mystem()
def g_name(name):
    global m
    res = m.analyze(name)
    fem, mal = 0, 0
    if len(res) == 6:
        res = res[2:]
    for i in res:
        if 'analysis' in i:
            #print (i['analysis'])
            if ',f,' in i['analysis'][0]['gr']:
                fem += 1
            elif ',m,' in i['analysis'][0]['gr']:
                mal += 1
    try:
        fam = name.split()[0]
    except IndexError:
        return None
    if fem > 0:
        return 'f'
    elif fem == 0 and mal > 1:
        return 'm'
    elif fam.endswith('ова') or fam.endswith('ева') or fam.endswith('ёва') or fam.endswith('ина') or fam.endswith('ына'):
        return 'f'
    else:
        return None

In [22]:
def load_one_url_page(link, field):
    headers = {'User-Agent': ua.random}
    r = session.get(link, headers=headers)
    html = r.text
    soup = BeautifulSoup(html, 'lxml')
    for i in soup.find_all('ul', attrs={'class':'list'})[0].find_all('a'):
        link = i['href']
        title = i.div.text
        year = i.span.text.split('/')[0]
        try:
            g = [g_name(name) for name in i.span.text.split('/')[1].split(', ')]
        except IndexError:
            g = ['no']
        if g.count('f') == len(g):
            g = 'f'
        elif g.count('m') == len(g):
            g = 'm'
        else:
            g = None
        if g is not None:
            cur.execute('INSERT INTO cyberleninka (field, name, g, title_url, title_text, year) \
            VALUES (?,?,?,?,?,?)', [field, i.span.text.split('/')[1], g, link, title, int(year)])
            
def get_adresses(start, start_page):
        for i in fields:
            clear_output()
            headers = {'User-Agent': ua.random}
            session = requests.session()
            main_link = 'https://cyberleninka.ru'+fields[i]
            r = session.get(main_link, headers=headers)
            html = r.text
            soup = BeautifulSoup(html, 'lxml')
            try:
                n_pages = int(soup.find_all('ul', attrs={'class':'paginator'})[0].find_all('a', attrs={'class':'icon'})[0]['href'].split('/')[-1])
            except:
                try:
                    n_pages = len(soup.find_all('ul', attrs={'class':'paginator'})[0].find_all('a'))
                except:
                    n_pages = 1
            print (i, n_pages)
            if i == start:
                zero = start_page
            else:
                zero = 0
            for n in tqdm(range(zero, n_pages)):
                link = main_link + '/' + str(n+1)
                load_one_url_page(link, i)
                db.commit()
                x = random.uniform(0.1,0.8)
                time.sleep(x)
            x = random.uniform(0.2,1.4)
            time.sleep(x)

In [28]:
a = list(fields.keys())

In [29]:
for i in a:
    if i < 'Педагогика':
        del fields[i]

In [23]:
db.rollback()

In [32]:
get_adresses('Педагогика', 3500)

Языкознание 3363


# Texts

In [21]:
def load_one_article(url_path, proxy=None):
    headers = {'User-Agent': ua.random}
    r = session.get('https://cyberleninka.ru/'+url_path, proxies=proxy, headers=headers)
    html = r.text
    soup = BeautifulSoup(html, 'lxml')
    try:
        keywords = ', '.join([i.text for i in soup.find_all('i', attrs={'itemprop':'keywords'})[0].find_all('span')]).lower()
    except IndexError:
        keywords = ''
    description = soup.find_all('p', attrs={'itemprop':'description'})
    if description:
        desc_ru = re.sub('<.*?>','',description[0].text)
        if len(description) > 1:
            desc_en = re.sub('<.*?>','',description[1].text)
        else:
            desc_en = ''
    else:
        desc_ru = ''
        desc_en = ''
    text = soup.find_all('div', attrs={'class':'ocr','itemprop':'articleBody'})[0].text.strip()
    return keywords, desc_ru, desc_en, text

def update_db(idx, keywords, desc_ru, desc_en, text):
    cur.execute('INSERT INTO texts (id, keywords, abstract_ru, abstract_en, text) \
            VALUES (?,?,?,?, ?)', [idx, keywords, desc_ru, desc_en, text])
    db.commit()

In [23]:
def download(fields, start):
    for j, field in enumerate(fields):
        clear_output()
        print (field)
        cur.execute('SELECT id, title_url FROM cyberleninka WHERE field = ?', (field,))
        if j == 0:
            links = cur.fetchall()[start:]
        else:
            links = cur.fetchall()
        k = 0
        n = 0
        for i in tqdm(links):
            
            if n % 100 == 0:
                proxies = list(get_proxies())[:20]
            if n % 37 == 0:
                x = random.choice(proxies)
                proxy = {'https':x, 'http': x}
                
            try:
                keywords, desc_ru, desc_en, text = load_one_article(i[1], proxy)
                update_db(i[0], keywords, desc_ru, desc_en, text)
            except IndexError:
                k += 1
                print (i[0])
            #except:
            #    print (i[0], 'ProxyError')
            #    k += 1
                    
            n += 1
            #x = random.uniform(2.5,9.7)
            #time.sleep(x)
        with open('errors', 'a') as f:
            f.write(field+'\t'+str(k)+'\n')

In [24]:
def get_proxies():
    #url = 'https://free-proxy-list.net/'
    url = 'https://free-proxy-list.net/anonymous-proxy.html'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:500]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies
proxies = list(get_proxies())
len(proxies)

63

In [25]:
proxies[0]

'154.0.15.166:42193'

In [29]:
session = requests.session()

In [ ]:
Атеизм
Религия

In [14]:
ConnectionRefusedError

ConnectionRefusedError

In [30]:
from requests.erors import ProxyError

ModuleNotFoundError: No module named 'requests.erors'

In [35]:
def download(fields, start):
    for j, field in enumerate(fields):
        clear_output()
        print (field)
        cur.execute('SELECT id, title_url FROM cyberleninka WHERE field = ?', (field,))
        if j == 0:
            links = cur.fetchall()[start:]
        else:
            links = cur.fetchall()
        k = 0
        n = 0
        for i in tqdm(links):                
            try:
                keywords, desc_ru, desc_en, text = load_one_article(i[1])
                update_db(i[0], keywords, desc_ru, desc_en, text)
            except IndexError:
                k += 1
                print (i[0])
            #except:
            #    print (i[0], 'ProxyError')
            #    k += 1
                    
            n += 1
            x = random.uniform(5.5,10.7)
            time.sleep(x)
        with open('errors', 'a') as f:
            f.write(field+'\t'+str(k)+'\n')

In [37]:
fields = ['Химия','Строительство','Геология','Физика','Вычислительная техника']
download(fields, 1615+87+37+208)

Геология


48970
48971
48972
48973
48974


KeyboardInterrupt: 